# EXPLORATORY DATA ANALYSIS

## Business Problem Understanding


Company X is expanding in to new industries to diversify its portfolio. They are particularly interested in purchasing and operating airplanes for commercial and private enterprises. However, they do not have enough knowledge on the potential risks of aircrafts.

**Task** 
- Determine which aircraft are the lowest risk for the company X to start this new endeavor. 
- Translate findings into actionable insights that the stake holders can use to help decide which aircraft to purchase. 

## The Data

The data is from the National Transportation Safety Board that includes aviation accident data from 1962 to 2023 about civil aviation accidents and selected incidents in the United States and international waters. 
The dataset was obtained from Kaggle. https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses 

### Loading the Data

In [2]:
# import necessary libraries
import pandas as pd

In [9]:
! pip install openpyxl  # add package to open excel files using python

   ---------------------------------------- 0.0/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/251.3 kB 262.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/251.3 kB 217.9 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/251.3 kB 297.7 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/251.3 kB 302.7 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/251.3 kB 374.1 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/251.3 kB 374.1 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/251.3 kB 425.3 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/251.3 kB 425.3 kB/s eta 0:00:01
   ----------------------------- -------- 194.6/251.3 kB 471.4 kB/s eta 0

In [95]:
df = pd.read_excel("AviationData.xlsx")
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaT
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,1996-09-19
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,2007-02-26
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,2000-09-12
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,1980-04-16


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                88889 non-null  object        
 1   Investigation.Type      88889 non-null  object        
 2   Accident.Number         88889 non-null  object        
 3   Event.Date              88889 non-null  datetime64[ns]
 4   Location                88837 non-null  object        
 5   Country                 88663 non-null  object        
 6   Latitude                34382 non-null  object        
 7   Longitude               34373 non-null  object        
 8   Airport.Code            50132 non-null  object        
 9   Airport.Name            52704 non-null  object        
 10  Injury.Severity         87889 non-null  object        
 11  Aircraft.damage         85695 non-null  object        
 12  Aircraft.Category       32287 non-null  object

In [97]:
# get the number of missing values
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

### Get relevant columns for our analysis

In [101]:
relevant_columns = [
    'Event.Date',
    'Investigation.Type',
    'Accident.Number',
    'Event.Date',
    'Country',
    'Injury.Severity',
    'Aircraft.damage',
    'Make',
    'Model',
    'Amateur.Built',
    'Number.of.Engines',
    'Engine.Type',
    'Purpose.of.flight',
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured',
    'Weather.Condition',
    'Publication.Date'
    
]

In [102]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [294]:
df2 = df.loc[:,relevant_columns]
df2.shape

(88889, 19)

In [295]:
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                     226
Injury.Severity            1000
Aircraft.damage            3194
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
Purpose.of.flight          6192
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Publication.Date          13771
dtype: int64

In [296]:
df2.Country.value_counts()

Country
United States                       82248
Brazil                                374
Canada                                359
Mexico                                358
United Kingdom                        344
                                    ...  
Seychelles                              1
Palau                                   1
Libya                                   1
Saint Vincent and the Grenadines        1
Turks and Caicos Islands                1
Name: count, Length: 219, dtype: int64

In [297]:
# Drop null rows in COUNTRY

df2 = df2[df2['Country'].notna()]

In [298]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88663 entries, 0 to 88888
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Date              88663 non-null  datetime64[ns]
 1   Investigation.Type      88663 non-null  object        
 2   Accident.Number         88663 non-null  object        
 3   Event.Date              88663 non-null  datetime64[ns]
 4   Country                 88663 non-null  object        
 5   Injury.Severity         87663 non-null  object        
 6   Aircraft.damage         85485 non-null  object        
 7   Make                    88601 non-null  object        
 8   Model                   88572 non-null  object        
 9   Amateur.Built           88561 non-null  object        
 10  Number.of.Engines       82587 non-null  float64       
 11  Engine.Type             81573 non-null  object        
 12  Purpose.of.flight       82478 non-null  object     

In [299]:
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                       0
Injury.Severity            1000
Aircraft.damage            3178
Make                         62
Model                        91
Amateur.Built               102
Number.of.Engines          6076
Engine.Type                7090
Purpose.of.flight          6185
Total.Fatal.Injuries      11390
Total.Serious.Injuries    12500
Total.Minor.Injuries      11923
Total.Uninjured            5903
Weather.Condition          4487
Publication.Date          13714
dtype: int64

In [300]:
# INJURY SEVERITY

df2['Injury.Severity'].mode()

0    Non-Fatal
Name: Injury.Severity, dtype: object

In [301]:
# AIRCRAFT DAMAGE

df2['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    64032
Destroyed      18542
Minor           2792
Unknown          119
Name: count, dtype: int64

In [302]:
# MAKE

df2 = df2[df2['Make'].notna()]

In [303]:
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                       0
Injury.Severity             983
Aircraft.damage            3161
Make                          0
Model                        49
Amateur.Built               100
Number.of.Engines          6028
Engine.Type                7040
Purpose.of.flight          6144
Total.Fatal.Injuries      11384
Total.Serious.Injuries    12491
Total.Minor.Injuries      11913
Total.Uninjured            5893
Weather.Condition          4450
Publication.Date          13711
dtype: int64

In [304]:
# MODEL
df2 = df2[df2['Model'].notna()]

In [305]:
df2.Model.value_counts()

Model
152                   2367
172                   1754
172N                  1161
PA-28-140              931
150                    828
                      ... 
CHALLENGER SPEC II       1
125-800A                 1
N2S-4                    1
PV2                      1
M-8 EAGLE                1
Name: count, Length: 12294, dtype: int64

In [306]:
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                       0
Injury.Severity             979
Aircraft.damage            3157
Make                          0
Model                         0
Amateur.Built                99
Number.of.Engines          6016
Engine.Type                7020
Purpose.of.flight          6132
Total.Fatal.Injuries      11376
Total.Serious.Injuries    12481
Total.Minor.Injuries      11905
Total.Uninjured            5889
Weather.Condition          4435
Publication.Date          13708
dtype: int64

In [307]:
# AMATEUR BUILT
df2 = df2[df2['Amateur.Built'].notna()]
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                       0
Injury.Severity             979
Aircraft.damage            3139
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines          5937
Engine.Type                6942
Purpose.of.flight          6067
Total.Fatal.Injuries      11310
Total.Serious.Injuries    12392
Total.Minor.Injuries      11812
Total.Uninjured            5820
Weather.Condition          4377
Publication.Date          13697
dtype: int64

In [308]:
# NUMBER OF ENGINES

df2['Number.of.Engines'].value_counts()

# 0.0 is an outlier. A plane can't have 0 engines

Number.of.Engines
1.0    69389
2.0    10993
0.0     1223
3.0      477
4.0      430
8.0        3
6.0        1
Name: count, dtype: int64

In [309]:
# use median to fill the missing values.
# Median and Mean are almost same and lie at 1
df2['Number.of.Engines'] = df['Number.of.Engines'].fillna(df2['Number.of.Engines'].median())
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88453 entries, 0 to 88888
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Date              88453 non-null  datetime64[ns]
 1   Investigation.Type      88453 non-null  object        
 2   Accident.Number         88453 non-null  object        
 3   Event.Date              88453 non-null  datetime64[ns]
 4   Country                 88453 non-null  object        
 5   Injury.Severity         87474 non-null  object        
 6   Aircraft.damage         85314 non-null  object        
 7   Make                    88453 non-null  object        
 8   Model                   88453 non-null  object        
 9   Amateur.Built           88453 non-null  object        
 10  Number.of.Engines       88453 non-null  float64       
 11  Engine.Type             81511 non-null  object        
 12  Purpose.of.flight       82386 non-null  object     

In [310]:
# Replace 'Number.of.Engines' with 1.0 where 'Number.of.Engines' is 0.0

df2['Number.of.Engines'] = df2['Number.of.Engines'].replace(0.0, 1.0)


df2['Number.of.Engines'].value_counts()

Number.of.Engines
1.0    76549
2.0    10993
3.0      477
4.0      430
8.0        3
6.0        1
Name: count, dtype: int64

In [313]:
# ENGINE TYPE

df2['Engine.Type'].value_counts()

# Unkown, None, UNK, are not valid engine types

Engine.Type
Reciprocating      69374
Turbo Shaft         3522
Turbo Prop          3379
Turbo Fan           2465
Unknown             2042
Turbo Jet            701
Geared Turbofan       12
Electric              10
LR                     2
NONE                   2
Hybrid Rocket          1
UNK                    1
Name: count, dtype: int64

In [316]:
# Unkown, None, UNK, are not valid engine types so we drop

df2 = df2[~df2['Engine.Type'].isin(['Unknown', 'NONE', 'UNK'])]


In [319]:
# drop the missing values
df2 = df2[df2['Engine.Type'].notna()]

In [321]:
df2.isna().sum()

Event.Date                    0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Country                       0
Injury.Severity             192
Aircraft.damage            1719
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
Purpose.of.flight          2808
Total.Fatal.Injuries      10355
Total.Serious.Injuries    11183
Total.Minor.Injuries      10499
Total.Uninjured            4924
Weather.Condition           910
Publication.Date          12097
dtype: int64

In [324]:
df2['Purpose.of.flight'].value_counts()

Purpose.of.flight
Personal                     46128
Instructional                 9980
Unknown                       5887
Aerial Application            4559
Business                      3749
Positioning                   1545
Other Work Use                1090
Ferry                          763
Aerial Observation             682
Public Aircraft                602
Executive/corporate            512
Flight Test                    370
Skydiving                      171
External Load                  103
Public Aircraft - Federal       95
Banner Tow                      95
Air Race show                   72
Public Aircraft - Local         71
Public Aircraft - State         62
Air Race/show                   48
Glider Tow                      37
Firefighting                    24
Air Drop                         8
PUBS                             2
ASHO                             2
PUBL                             1
Name: count, dtype: int64

In [323]:
df2['Purpose.of.flight'].isna().sum()

2808

In [327]:
# we can drop the missing values in Purpose.of.flight
df2 = df2[df2['Purpose.of.flight'].notna()]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76658 entries, 0 to 88767
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Date              76658 non-null  datetime64[ns]
 1   Investigation.Type      76658 non-null  object        
 2   Accident.Number         76658 non-null  object        
 3   Event.Date              76658 non-null  datetime64[ns]
 4   Country                 76658 non-null  object        
 5   Injury.Severity         76635 non-null  object        
 6   Aircraft.damage         75548 non-null  object        
 7   Make                    76658 non-null  object        
 8   Model                   76658 non-null  object        
 9   Amateur.Built           76658 non-null  object        
 10  Number.of.Engines       76658 non-null  float64       
 11  Engine.Type             76658 non-null  object        
 12  Purpose.of.flight       76658 non-null  object     

In [ ]:
df2. 